In [ ]:
# Cell 1 – Imports et Configuration
# ───────────────────────────────────────────────────────────────────────────
from pathlib import Path
import shutil
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image

# Chemins d’entrée et de sortie
ROOT_IN   = Path(r"C:\Users\PC\Desktop\dataset_final")       # dossier source
ROOT_OUT  = ROOT_IN.parent / "dataset_final_pp"              # dossier de sortie
CSV_IN    = ROOT_IN / "index.csv"                            # index initial
CSV_OUT   = ROOT_OUT / "index_pp.csv"                         # index prétraité

# Paramètres de traitement
TARGET_DIM   = 1024        # dimension carrée après resize
APPLY_ZSCORE = True        # si False, on fait un min-max [0-255]

# Configuration CLAHE (Contrast Limited Adaptive Histogram Equalization)
CLAHE_CLIP = 2.0
CLAHE_GRID = (8, 8)
clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP, tileGridSize=CLAHE_GRID)


In [ ]:
# Cell 2 – Fonctions Utilitaires Simples
# ───────────────────────────────────────────────────────────────────────────
def load_gray(path):
    """
    Charge une image en niveaux de gris à partir du chemin donné.
    Retourne un tableau NumPy (dtype=uint8).
    """
    return cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)


def save_png(image: np.ndarray, path):
    """
    Sauvegarde un tableau NumPy en tant que fichier PNG.
    """
    Image.fromarray(image).save(path)


In [ ]:
# Cell 3 – Détection et Crop de la Région Mammaire (BBox)
# ───────────────────────────────────────────────────────────────────────────
def breast_bbox(img: np.ndarray):
    """
    Renvoie la bounding box (x, y, w, h) du tissu mammaire
    en utilisant une seuillage Otsu + composant connecté le plus grand.
    """
    # 1) Seuillage Otsu
    thr, _ = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    tissue = (img > thr).astype(np.uint8)

    # 2) Composants connectés
    nb, lbl, stats, _ = cv2.connectedComponentsWithStats(tissue, connectivity=8)
    if nb <= 1:
        # Si un seul composant (le fond), on prend toute l'image
        return 0, 0, img.shape[1], img.shape[0]

    # On cherche l'index du plus grand composant (hors fond)
    largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    x, y, w, h = stats[largest, :4]
    return x, y, w, h


def pad_to_square(im: np.ndarray, ma: np.ndarray):
    """
    Pad l'image et le masque pour en faire un carré avant resize.
    Retourne (im_padded, mask_padded).
    """
    h, w = im.shape
    size = max(h, w)
    pad_b = size - h
    pad_r = size - w
    im_p = cv2.copyMakeBorder(im, 0, pad_b, 0, pad_r, cv2.BORDER_CONSTANT, value=0)
    ma_p = cv2.copyMakeBorder(ma, 0, pad_b, 0, pad_r, cv2.BORDER_CONSTANT, value=0)
    return im_p, ma_p


In [ ]:
# Cell 4 – Nettoyage du Masque (OpenCV only)
# ───────────────────────────────────────────────────────────────────────────
def clean_mask(mask: np.ndarray, min_area: int = 64):
    """
    Effectue un closing morphologique, remplit les trous, puis supprime
    les petits artefacts (composants de surface < min_area).
    """
    # 1) Binarisation
    mask_bin = (mask > 127).astype(np.uint8)

    # 2) Fermeture (closing) morphologique
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask_bin = cv2.morphologyEx(mask_bin, cv2.MORPH_CLOSE, kernel)

    # 3) Remplissage des trous (flood-fill du fond puis inversion)
    flood = mask_bin.copy()
    h, w = flood.shape
    cv2.floodFill(flood, None, (0, 0), 255)   # remplit l'extérieur
    holes = cv2.bitwise_not(flood)
    mask_bin = cv2.bitwise_or(mask_bin * 255, holes)

    # 4) Suppression des petits composants
    nb, lbl, stats, _ = cv2.connectedComponentsWithStats(mask_bin, connectivity=8)
    cleaned = np.zeros_like(mask_bin)
    for i in range(1, nb):
        if stats[i, cv2.CC_STAT_AREA] >= min_area:
            cleaned[lbl == i] = 255
    return cleaned.astype(np.uint8)


In [ ]:
# Cell 5 – Prétraitement Complet d’une Paire Image + Masque
# ───────────────────────────────────────────────────────────────────────────
def preprocess_pair(img_path: str, mask_path: str):
    """
    Charge l'image et son masque depuis disque, puis :
      1) détecte et croppe le tissu mammaire
      2) pad en carré + resize à (TARGET_DIM, TARGET_DIM)
      3) applique CLAHE pour améliorer le contraste
      4) normalise (z-score ou min-max)
      5) nettoie le masque (closing + suppression petits artefacts)
    Retourne (img_preprocessed, mask_cleaned) sous forme de tableaux uint8.
    """
    # 1) Chargement et crop du tissu
    img  = load_gray(img_path)
    mask = load_gray(mask_path)
    x, y, w, h = breast_bbox(img)
    img  = img[y:y+h, x:x+w]
    mask = mask[y:y+h, x:x+w]

    # 2) Pad → carré et resize
    img, mask = pad_to_square(img, mask)
    img  = cv2.resize(img,  (TARGET_DIM, TARGET_DIM), interpolation=cv2.INTER_AREA)
    mask = cv2.resize(mask, (TARGET_DIM, TARGET_DIM), interpolation=cv2.INTER_NEAREST)

    # 3) CLAHE sur l'image
    img = clahe.apply(img)

    # 4) Normalisation
    if APPLY_ZSCORE:
        m, s = img.mean(), img.std() + 1e-8
        img = ((img - m) / s * 16 + 128).clip(0, 255).astype(np.uint8)
    else:
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

    # 5) Nettoyage du masque
    mask = clean_mask(mask)

    return img, mask


In [ ]:
# Cell 6 – Boucle Principale sur le Dataset
# ───────────────────────────────────────────────────────────────────────────
def main_preprocessing():
    """
    Parcourt chaque ligne de CSV_IN, appelle preprocess_pair(), 
    enregistre les images traitées dans ROOT_OUT/<patient_id>/ et 
    produit un nouveau index CSV_OUT.
    """
    # 1) Création du dossier de sortie
    ROOT_OUT.mkdir(exist_ok=True)

    # 2) Lecture de l'index initial
    df_in = pd.read_csv(CSV_IN)

    # 3) Liste pour collecter les nouveaux enregistrements
    out_records = []

    # 4) Parcours des lignes
    for _, row in tqdm(df_in.iterrows(), total=len(df_in), desc="Pre-processing"):
        img_pp, mask_pp = preprocess_pair(row["image_png"], row["mask_png"])

        pid, view = row["patient_id"], row["view"]
        out_dir = ROOT_OUT / pid
        out_dir.mkdir(exist_ok=True)

        img_fn  = out_dir / f"image_{view}.png"
        mask_fn = out_dir / f"mask_{view}.png"

        save_png(img_pp,  img_fn)
        save_png(mask_pp, mask_fn)

        out_records.append({
            "patient_id": pid,
            "view": view,
            "image_png": img_fn.as_posix(),
            "mask_png":  mask_fn.as_posix()
        })

    # 5) Copie éventuelle des fichiers cliniques
    for extra in ("clinical_data.xlsx", "metadata.csv"):
        src = ROOT_IN / extra
        if src.exists():
            shutil.copy(src, ROOT_OUT / extra)

    # 6) Écriture du nouvel index
    pd.DataFrame(out_records).to_csv(CSV_OUT, index=False, encoding="utf-8")
    print(f"✅  {len(out_records)} paires traitées — index ➜ {CSV_OUT}")

# Pour lancer dans Jupyter :
# main_preprocessing()
